### BRDF from Brightness Observation
This code infers the BRDFs of the Starlink v1.5's primary surfaces
by reducing the error between observation and model prediction. The
BRDFs are assumed to be Phong BRDFs with 3 free parameters.

In [23]:
# Imports
import pandas as pd
import numpy as np
import astropy.time
import astropy.coordinates
import scipy.optimize
from lumos.geometry import Surface
import lumos.brdf.library

In [24]:
data = pd.read_csv("/Users/physarah/Desktop/updated_model_with_area.csv", comment = "#")
data.columns

Index(['Unnamed: 0.5', 'Unnamed: 0.4', 'Unnamed: 0.3', 'Unnamed: 0.2',
       'Unnamed: 0.1', 'Unnamed: 0', 'source_mean', 'source_median',
       'source_sum', 'source_rate', 'sky_mean', 'sky_median', 'sky_sum',
       'sky_rate', 'target_mean', 'target_median', 'target_sum', 'target_rate',
       'sc_area', 'bk_area', 'exposure_time', 'datetimes_sat', 'gains_sat',
       'airmass', 'm_inst', 'm_inst_sky', 'calibed_mags', 'altitude',
       'azimuth', 'utc', 'zenith_angle', 'km_distance', 'sunsep', 'sat_name',
       'norad_id', 'v_type', 'tle_filename', 'tle1', 'tle2', 'tle3',
       'altitude2', 'azimuth2', 'forest_time', 'line_distance', 'sat_height',
       'longitude', 'latitude', 'forrest_magnitudes_lambertian_09',
       'forrest_magnitudes_green', 'forrest_magnitudes', 'sun_alt', 'sun_az',
       'updated_prediction_mag', 'updated_pred_mag'],
      dtype='object')

In [25]:
# Read Sarah's database
data = data[data['v_type']=='V2.0 mini']
satellite_heights = data['sat_height'].to_numpy()
observation_times = data['datetimes_sat'].to_numpy()
satellite_altitudes = data['altitude2'].to_numpy()
satellite_azimuths = data['azimuth2'].to_numpy()
measured_magnitudes = data['updated_pred_mag'].to_numpy()
sun_alts = data['sun_alt'].to_numpy()
sun_azs = data['sun_az'].to_numpy()
mount_lemmon = astropy.coordinates.EarthLocation(lat = -33.7703, lon = 151.1112)

In [26]:
# # Converts from observation time to Sun's altitude and azimuth
# sun_alts = []
# sun_azs = []
# for time in observation_times:
#     obs_time = astropy.time.Time(time, format = 'isot')
#     sun_alt, sun_az = lumos.calculator.get_sun_alt_az(obs_time, mount_lemmon)
#     sun_alts.append(sun_alt)
#     sun_azs.append(sun_az)

In [30]:
# Main function to minimize RMS between model prediction and observations
def optimization(params):

    p1, p2, p3, p4, p5, p6 = params

    calculated_intensities = np.zeros_like(measured_magnitudes)

    # Sets up surfaces of satellite, one for the chassis and one for the solar array
    # Phong parameters, p1 - p6 vary
    surfaces = [
        Surface(1, np.array([0, 0, -1]), lumos.brdf.library.PHONG(p1, p2, p3)),
        Surface(1, np.array([0, 1, 0]), lumos.brdf.library.PHONG(p4, p5, p6))
        ]

    # Calculates predicted AB Mag for every observation
    for i, (sat_alt, sat_az, sat_h, sun_alt, sun_az) \
        in enumerate(zip(satellite_altitudes, satellite_azimuths, satellite_heights, sun_alts, sun_azs)):

        calculated_intensities[i] = lumos.calculator.get_intensity_observer_frame(
            surfaces,
            sat_h,
            sat_alt,
            sat_az,
            sun_alt,
            sun_az,
            include_sun = True,
            include_earthshine = False,
            earth_brdf = lumos.brdf.library.PHONG(0.53, 0.28, 7.31),
            earth_panel_density=300
            )
    
    calculated_magnitudes = lumos.conversions.intensity_to_ab_mag(calculated_intensities)

    # Return RMS between prediction and actual observation
    error = calculated_magnitudes - measured_magnitudes
    rms = np.sqrt( np.mean( error**2 ) )
    return rms

sol = scipy.optimize.minimize(
    optimization,
    x0 = (1, 1, 1, 1, 1, 1),
    bounds = ( (0, 20), (0, 20), (0, 200), 
               (0, 20), (0, 20), (0, 200)) )

Kd1, Ks1, n1, Kd2, Ks2, n2 = sol.x
print(f"RMS = {sol.fun:0.3f}")

RMS = 0.344


In [31]:
print("Chassis Inferred BRDF Parameters")
print("------------------------------------")
print(f"Kd = {Kd1:0.2f}")
print(f"Ks = {Ks1:0.2f}")
print(f"n = {n1:0.2f}")

print("Solar Array Inferred BRDF Parameters")
print("------------------------------------")
print(f"Kd = {Kd2:0.2f}")
print(f"Ks = {Ks2:0.2f}")
print(f"n = {n2:0.2f}")

Chassis Inferred BRDF Parameters
------------------------------------
Kd = 1.00
Ks = 1.00
n = 1.00
Solar Array Inferred BRDF Parameters
------------------------------------
Kd = 3.75
Ks = 0.89
n = 29.59
